In [4]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [5]:
df = pd.read_csv("data/diabetes_prediction.csv")
df.pop("diabetes")

0        0
1        0
2        0
3        0
4        0
        ..
96141    0
96142    0
96143    0
96144    0
96145    0
Name: diabetes, Length: 96146, dtype: int64

In [6]:
columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'gender': 'Female',
 'age': 20.0,
 'hypertension': 0,
 'heart_disease': 0,
 'smoking_history': 'never',
 'bmi': 27.32,
 'HbA1c_level': 6.6,
 'blood_glucose_level': 85}

In [7]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://model-diabetes-production.up.railway.app/v1/models/diabetes-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "Normal"
    else:
        return "Diabetes"

In [8]:
# contoh hasil prediksi

make_predictions(inputs)

'Normal'